In [1]:
import sys
import pandas as pd
import re
from datetime import datetime

reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
file_content = open('../data/test.txt', 'r').read().splitlines()

In [4]:
df = pd.DataFrame(columns=['user', 'action', 'text', 'timestamp', 'date', 'month', 'day', 'hour'])

message_action = 'message'
media_action = 'media'
create_action = 'create'
title_action = 'title_change'
contact_action = 'contact_change'
icon_action = 'icon_change'
join_action = 'join'
remove_action = 'remove'
leave_action = 'left'

media_finder = '<Media omitted>'
create_finder = 'created group'
title_finder = 'changed the subject'
contact_finder = 'changed from'
icon_finder = ['changed', 'icon']
join_finder = 'added'
remove_finder = 'removed'
leave_finder = 'left'

title_actions = [create_action, title_action]
msg_actions = [message_action, media_action]
lead_actions = [create_action, icon_action, title_action, join_action, remove_action]

In [5]:
for message in file_content:
    msg_regex = re.compile('(^\d+\/\d+\/\d+, \d+:\d+) - (.*)')
    splitted_msg = msg_regex.search(message)
    if splitted_msg is not None:
        time_string = datetime.strptime(splitted_msg.group(1), '%m/%d/%y, %H:%M')

        timestamp = time_string.strftime('%x %X')
        date = time_string.strftime('%x')
        month = time_string.strftime('%B')
        day = time_string.strftime('%A')
        hour = time_string.strftime('%I %p')

        parsed_content = splitted_msg.group(2).split(':', 1)

        if len(parsed_content) > 1:
            user = parsed_content[0]
            text = parsed_content[1].strip()
            action = media_action if text == media_finder else message_action

        else:
            if title_finder in parsed_content[0]:
                user = parsed_content[0].split(title_finder)[0]
                text = parsed_content[0].split('to', 1)[-1].strip()
                action = title_action

            elif all(x in parsed_content[0] for x in icon_finder):
                user = parsed_content[0].split('changed')[0]
                text = None
                action = icon_action

            elif contact_finder in parsed_content[0]:
                user = parsed_content[0].split(contact_finder)[0]
                text = None
                action = contact_action

            elif join_finder in parsed_content[0]:
                sub_content = parsed_content[0].split(join_finder)
                user = sub_content[0]
                text = sub_content[1].strip()
                action = join_action

            elif remove_finder in parsed_content[0]:
                sub_content = parsed_content[0].split(join_finder)
                user = sub_content[0]
                text = sub_content[1].strip()
                action = remove_action

            elif leave_finder in parsed_content[0]:
                user = parsed_content[0].split(leave_finder)[0]
                text = None
                action = leave_action

            elif create_finder in parsed_content[0]:
                sub_content = parsed_content[0].split(create_finder)
                user = sub_content[0].strip()
                text = sub_content[-1].strip()
                action = create_action

        df.loc[len(df)] = [user, action, text, timestamp, date, month, day, hour]

    else:
        pass

In [6]:
def strip_time(timestamp):
    return datetime.strptime(timestamp, "%x %X")

def current_time():
    return datetime.now().strftime('%x %X')

def get_title_period(data):
    h = {}
    length = len(data)
    for i in range(length):
        prev_time = strip_time(data.iloc[i].timestamp)
        new_time = (strip_time(data.iloc[i+1].timestamp)) if i < length-1 else (strip_time(current_time()))
        h[data.iloc[i].text] = (new_time - prev_time).seconds
    return h

In [7]:
date_activity = df['date'].value_counts()
user_activity = df['user'].value_counts()

active_date = date_activity.idxmax()
active_user = user_activity.idxmax()
active_day = df['day'].value_counts().idxmax()
active_hour = df['hour'].value_counts().idxmax()

media_share = df[df.action == media_action].user.value_counts().to_dict()
icon_change = df[df.action == icon_action].user.value_counts().idxmax()
contact_change = df[df.action == contact_action].user.value_counts().idxmax()
creator = df[df.action == create_action].user[0]

title_data = df.query('action in @title_actions')
title_change = title_data.user.value_counts().idxmax()

all_titles = title_data.text.values
title_period = get_title_period(title_data)
stable_title = max(title_period, key=title_period.get)

message_data = df.query('action in @msg_actions')
total_messages = len(message_data)
talker = message_data.user.value_counts().idxmax()
silent_killer = message_data.user.value_counts().idxmin()

supervisor = df.query('action in @lead_actions').user.value_counts().idxmax()

In [29]:
# date_activity
# user_activity
# active_date
# active_user
# active_day
# active_hour
# media_share
# icon_change
# contact_change
# creator
# title_change
# all_titles
# stable_title
# total_messages
# talker
# silent_killer
# supervisor

'TCS Ethiraj '

In [8]:
url_regex = re.compile('^(?:(?:https?|ftp)://)(?:\S+(?::\S*)?@)?(?:(?:[1-9]\d?|1\d\d|2[01]\d|22[0-3])(?:\.(?:1?\d{1,2}|2[0-4]\d|25[0-5])){2}(?:\.(?:[1-9]\d?|1\d\d|2[0-4]\d|25[0-4]))|(?:(?:[a-z\u00a1-\uffff0-9]+-?)*[a-z\u00a1-\uffff0-9]+)(?:\.(?:[a-z\u00a1-\uffff0-9]+-?)*[a-z\u00a1-\uffff0-9]+)*(?:\.(?:[a-z\u00a1-\uffff]{2,})))(?::\d{2,5})?(?:/[^\s]*)?$')
text_messages = df[df.action == message_action].text.values
all_words = [word for item in text_messages for word in item.split()]

urls = [word for word in all_words if re.match(url_regex, word)]
stop_words = ['at', 'on', 'the']

[word for word in all_words if (word not in stop_words) and (word not in urls)] 

['Join',
 'me',
 'Elixir-Ruby',
 'joint',
 'meet',
 '9',
 'Jan',
 '2:00',
 'pm:',
 'Today',
 'n',
 '2mrw',
 'onnuke',
 'vaku',
 'illa',
 '\xf0\x9f\x98\x82\xf0\x9f\x98\x82',
 '\xf0\x9f\x98\x82\xf0\x9f\x98\x82',
 'Guy1',
 ':',
 'enga',
 'vela',
 'pakra',
 'pa?',
 'still',
 'i',
 'dont',
 'get',
 'joke',
 '\xe0\xae\x87\xe0\xae\xb5\xe0\xae\xa9\xe0\xaf\x8d',
 '\xe0\xae\x89\xe0\xae\xb0\xe0\xaf\x81\xe0\xae\xb5\xe0\xae\xae\xe0\xaf\x8d',
 '\xe0\xae\xa4\xe0\xaf\x87\xe0\xae\xb5\xe0\xaf\x88\xe0\xae\xaf\xe0\xae\xbf\xe0\xae\xb2\xe0\xaf\x8d\xe0\xae\xb2\xe0\xaf\x88,',
 '\xe0\xae\xa8\xe0\xae\xbf\xe0\xae\xb4\xe0\xae\xb4\xe0\xaf\x8d\xe0\xae\xa4\xe0\xaf\x87\xe0\xae\xb5\xe0\xaf\x88\xe0\xae\xaf\xe0\xae\xbf\xe0\xae\xb2\xe0\xaf\x8d\xe0\xae\xb2\xe0\xaf\x88',
 '\xe0\xae\x87\xe0\xae\xb5\xe0\xae\xa9\xe0\xaf\x81\xe0\xae\x9f\xe0\xaf\x88\xe0\xae\xaf']